In [1]:
!pip uninstall -y protobuf tensorflow-metadata
!pip install protobuf==3.20.3 tensorflow-metadata==1.13.1


Found existing installation: protobuf 6.33.0
Uninstalling protobuf-6.33.0:
  Successfully uninstalled protobuf-6.33.0
Found existing installation: tensorflow-metadata 1.17.2
Uninstalling tensorflow-metadata-1.17.2:
  Successfully uninstalled tensorflow-metadata-1.17.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.

# imports

In [2]:
import os, numpy as np, tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.metrics import classification_report


2025-12-15 01:26:09.278658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765761969.464560      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765761969.524939      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Config

In [3]:
BASE_DIR = "/kaggle/working/mepe"
MODEL_DIR = f"{BASE_DIR}/models/text_emotion"

TEXT_MODEL = "distilbert-base-uncased"
MAX_LEN = 96
BATCH_SIZE = 32
LR = 2e-5
EPOCHS = 3
NUM_LABELS = 28


# Load Dataset

In [4]:
go = load_dataset("go_emotions")

README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

# Convert multi-label → single label 

In [5]:
# if it returning multiple, we take the most dominant one)
def pick_single_label(example):
    example["labels"] = example["labels"][0]
    return example

go = go.map(pick_single_label)


Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

# Tokenization ( no tf tensors here )

In [6]:
tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL)

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

go = go.map(tokenize, batched=True, remove_columns=["text"])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

# Build tf.data.Dataset 

In [7]:
def make_tf_dataset(split, shuffle=False):
    x = {
        "input_ids": tf.constant(split["input_ids"], dtype=tf.int32),
        "attention_mask": tf.constant(split["attention_mask"], dtype=tf.int32)
    }
    y = tf.constant(split["labels"], dtype=tf.int32)

    ds = tf.data.Dataset.from_tensor_slices((x, y))
    if shuffle:
        ds = ds.shuffle(10000)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_ds = make_tf_dataset(go["train"], shuffle=True)
val_ds   = make_tf_dataset(go["validation"])


I0000 00:00:1765762008.417068      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765762008.417674      20 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


# Build Model

In [8]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    TEXT_MODEL,
    num_labels=NUM_LABELS,
    from_pt=True
)


pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSeq

# Compile (TF 2.15 SAFE)

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

model.summary()


Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  21532     
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66975004 (255.49 MB)
Trainable params: 66975004 (255.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Train

In [10]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Epoch 1/3


I0000 00:00:1765762040.259062     110 service.cc:148] XLA service 0x797538e841a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765762040.259801     110 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765762040.259818     110 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765762040.329950     110 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1765762040.454000     110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1357/1357 [==============================] - 481s 335ms/step - loss: 1.7906 - accuracy: 0.5066 - val_loss: 1.4158 - val_accuracy: 0.5772
Epoch 2/3
1357/1357 [==============================] - 460s 339ms/step - loss: 1.3257 - accuracy: 0.5970 - val_loss: 1.3671 - val_accuracy: 0.5813
Epoch 3/3
1357/1357 [==============================] - 460s 339ms/step - loss: 1.1258 - accuracy: 0.6505 - val_loss: 1.3894 - val_accuracy: 0.5798


# Evaluation

In [11]:
# with only metrics: (f1), since this is only a classifaication

y_true, y_pred = [], []

for batch in val_ds:
    logits = model(batch[0]).logits
    preds = tf.argmax(logits, axis=-1)
    y_pred.extend(preds.numpy())
    y_true.extend(batch[1].numpy())

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.71      0.77      0.74       488
           1       0.73      0.81      0.77       297
           2       0.49      0.55      0.52       192
           3       0.31      0.33      0.32       247
           4       0.48      0.27      0.34       355
           5       0.41      0.52      0.46       138
           6       0.50      0.38      0.43       136
           7       0.45      0.57      0.50       205
           8       0.63      0.45      0.53        64
           9       0.41      0.34      0.37       129
          10       0.44      0.27      0.33       246
          11       0.48      0.55      0.51        74
          12       0.71      0.36      0.48        28
          13       0.38      0.23      0.29        78
          14       0.68      0.57      0.62        74
          15       0.87      0.82      0.84       297
          16       0.00      0.00      0.00        10
          17       0.59    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Save model

In [12]:
model.save_pretrained(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)

os.listdir(MODEL_DIR)


['vocab.txt',
 'tf_model.h5',
 'tokenizer.json',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'config.json']

# Inference functions

In [13]:
def predict_text_emotion(text):
    inputs = tokenizer(
        text,
        return_tensors="tf",
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )
    logits = model(**inputs).logits
    probs = tf.nn.softmax(logits, axis=-1)
    return probs.numpy()[0]


# Test

In [14]:
predict_text_emotion("I feel very sad and disappointed today")[:5]


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


array([0.00058652, 0.00053433, 0.00309452, 0.00734428, 0.00372874],
      dtype=float32)

# saving the model in HF format

In [15]:
import os

HF_TEXT_DIR = "/kaggle/working/mepe/models/text_emotion_hf"
os.makedirs(HF_TEXT_DIR, exist_ok=True)

model.save_pretrained(HF_TEXT_DIR)
tokenizer.save_pretrained(HF_TEXT_DIR)

print("HF text model saved to:", HF_TEXT_DIR)
print("Files:", os.listdir(HF_TEXT_DIR))



HF text model saved to: /kaggle/working/mepe/models/text_emotion_hf
Files: ['vocab.txt', 'tf_model.h5', 'tokenizer.json', 'tokenizer_config.json', 'special_tokens_map.json', 'config.json']


# checking wether the model is loading or not? ( for later phases )

In [16]:
import os
os.listdir("/kaggle/working/mepe/models/text_emotion_hf")



['vocab.txt',
 'tf_model.h5',
 'tokenizer.json',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'config.json']